# Study Experiment Design Generation

This notebook analyzes the characteristics of different space-filling experiment design generation techniques provide by raxpy for different input spaces.

In [1]:
from typing import NamedTuple, Callable, List, Tuple

import matplotlib.pyplot as plt
import numpy as np

import raxpy
import raxpy.spaces as s
import raxpy.spaces.complexity as c
import raxpy.does.lhs as lhs_doe
import raxpy.does.random as random_doe
import raxpy.does.measure as measure
from raxpy.does import plots
import raxpy.does.doe 
from raxpy.does.doe import EncodingEnum

import importlib

importlib.reload(raxpy.does.doe)
plt.style.use('ggplot')


class DoeTuple(NamedTuple):
    design:lhs_doe.DesignOfExperiment
    measurement_set:measure.DesignMeasurementSet


class DesignStrategyResults(NamedTuple):
    strategy:Callable
    name:str
    designs:List[DoeTuple]


## Create Helper Functions

In [2]:
def compute_sub_space_allocations(doe, sub_spaces):
    actual_counts = {sub_space:0 for sub_space in sub_spaces}

    # determine the sub-space each data-point belongs to
    def map_point(point):
        active_dim_ids = []

        for dim_id, column_index in doe.input_set_map.items():
            if ~np.isnan(point[column_index]):
                active_dim_ids.append(dim_id)

        active_dim_ids.sort()
        actual_counts[tuple(a for a in active_dim_ids)] += 1

    # compute the subspace each point belongs to
    for point in doe.decoded_input_sets:
        map_point(point)
    
    return actual_counts
        
def meets_portions(doe, expected_counts):
    actual_counts = compute_sub_space_allocations(doe,expected_counts.keys())

    for key in expected_counts.keys():
        if expected_counts[key] != actual_counts[key]:
            print(actual_counts)
            return False

    return True

def generate_designs(strategies:List[DesignStrategyResults], space: s.InputSpace, number_of_designs: int = 10, number_of_points: int = 100, target_full_sub_space_portions=None):
    for _, strategy in enumerate(strategies):
        print(f"Generating designs for strategy: '{strategy[1]}'")
        design_count = 0
        while design_count < number_of_designs:
            doe = strategy[0](space, number_of_points)

            if target_full_sub_space_portions is None or meets_portions(doe, target_full_sub_space_portions):
                # note that we use decoded values for measurement, 
                # This allows comparison with all stratgies since some 
                # generate designs work in decoded space
                # All input spaces use floats with 0 to 1 bounds so 
                # dimensional range bias not an issue.
                measurement_set = measure.measure_with_all_metrics(doe, encoding=EncodingEnum.NONE)
                strategy[2].append(DoeTuple(doe,measurement_set))
                design_count += 1
                print(f"Created design {design_count} for {strategy[1]}")
            else:
                print("Skipping design")
            

In [3]:
def plot_fullsubspace_target_portions(space: s.InputSpace, number_of_points: int = 100):
    subspaces = space.derive_full_subspaces()
    
    values = c.compute_subspace_portions(space, subspaces)
    
    fig, axs = plt.subplots(nrows=1, ncols=1, figsize=(9, 4))

    rects = axs.bar(x=list(i for i in range(len(subspaces))),height=values, tick_label=list(", ".join(subspace) for subspace in subspaces))

    axs.bar_label(rects, labels=list(f"{int(value*100)}% - {round(value*number_of_points)}" for value in values))

    axs.set_ylabel("Portion Percentage")

    axs.set_title(f'Target Portions for {number_of_points} points')
    plt.xticks(rotation=45)
    plt.show()

In [4]:

def dict_to_hashable_tuple(d):
    return tuple(sorted(d.items()))

class DesignAssessmentGroups:

    def __init__(self, space, strategy_names):
        self.design_allocations = {}
        self.space = space
        self.strategy_names = strategy_names
        self.sub_spaces = tuple(tuple(l) for l in space.derive_full_subspaces())

    def add_design(self,strategy_name, doe_tuple:DoeTuple):
        
        allocation = dict_to_hashable_tuple(
            compute_sub_space_allocations(doe_tuple.design, self.sub_spaces)
        )
        if allocation not in self.design_allocations:
            self.design_allocations[allocation] = {name:[] for name in self.strategy_names}

        self.design_allocations[allocation][strategy_name].append(doe_tuple)
    
    def print_allocation_counts(self):

        for full_sub_space, design_allocation in self.design_allocations.items():
            
            print(f"{full_sub_space} - {', '.join(str(len(d)) for d in design_allocation.values())}")

    def generate_allocation_point_differences(self, target_sub_space_allocations):
        count_differences = {name:[] for name in self.strategy_names}
        for full_sub_space, design_allocation in self.design_allocations.items():
            # compute point count differences from target

            diff = 0
            for sub_spaces, actual_count in full_sub_space:
                target_count = target_sub_space_allocations[sub_spaces]
                diff += abs(target_count - actual_count)

            diff = diff/2.0 # avoid double counting
            for strat_name, doe_tuples in design_allocation.items():
                for _ in range(len(doe_tuples)):
                    count_differences[strat_name].append(diff)
        
        fig, axs = plt.subplots(nrows=1, ncols=1, figsize=(9, 4))

        axs.violinplot(list(v for v in count_differences.values()),
                        showmeans=False,
                        showmedians=True)
        axs.set_xticks(list(i+1 for i in range(len(self.strategy_names))))
        axs.set_xticklabels(self.strategy_names)

        axs.set_title(f'Allocation Point Count Differences from Target')

        plt.show()

def split_designs_by_subspace_allocations(strategies:List[DesignStrategyResults], space:s.InputSpace) -> DesignAssessmentGroups:

    
    assessment_group = DesignAssessmentGroups(space, strategy_names=list(s.name for s in strategies))

    
    for strategy in strategies:

        for design in strategy.designs:
            # determine the allocations to sub-spaces
            assessment_group.add_design(strategy.name, design)
    

    return assessment_group


In [5]:
def get_sub_space_measurements(strategies:List[DesignStrategyResults], dim_list, metric=measure.METRIC_DISCREPANCY):
    results = []
    for strategy in strategies:
        design_results = []
        for doe_tuple in strategy.designs:
            assessment = doe_tuple.measurement_set.get_full_sub_design_measurements(dim_list)
            if assessment is not None:
                if metric in assessment.measurements:
                    design_results.append(assessment.measurements[metric])
                else:
                    print(f"Skipping design for strategy '{strategy.name}' since it does not metric '{metric}' ")
        results.append(design_results)

    return results

In [6]:
def plot_sub_space_assessments(strategies:List[DesignStrategyResults], dim_list, metric=measure.METRIC_DISCREPANCY):
    assessment_values = get_sub_space_measurements(strategies, dim_list, metric)

    assessmentic_data = [[t for t in assessment_values[i] ] for i in range(len(strategies))]
    fig, axs = plt.subplots(nrows=1, ncols=1, figsize=(9, 4))

    axs.violinplot(assessmentic_data,
                    showmeans=False,
                    showmedians=True)
    axs.set_xticks(list(i+1 for i in range(len(strategies))))
    axs.set_xticklabels(list(strategy.name for strategy in strategies))

    axs.set_title(f'{metric} for {", ".join(dim_list)} full-sub-design (smaller the better)')

    plt.show()
    pass

In [7]:
def generate_basic_plots(strategies:List[DesignStrategyResults]):
    
    assessmentic_data = [[t.measurement_set.measurements[measure.METRIC_WEIGHTED_DISCREPANCY] for t in strategy.designs ] for strategy in strategies]
    fig, axs = plt.subplots(nrows=1, ncols=1, figsize=(9, 4))

    axs.violinplot(assessmentic_data,
                    showmeans=False,
                    showmedians=True)
    axs.set_title('Weighted Discrepancies (smaller the better)')
    axs.set_xticks(list(i+1 for i in range(len(strategies))))
    axs.set_xticklabels(list(strategy.name for strategy in strategies))

    plt.show()

    assessmentic_data = [[t.measurement_set.measurements[measure.METRIC_WHOLE_MIN_POINT_DISTANCE] for t in strategy.designs ] for strategy in strategies]
    fig, axs = plt.subplots(nrows=1, ncols=1, figsize=(9, 4))

    axs.violinplot(assessmentic_data,
                    showmeans=False,
                    showmedians=True)
    axs.set_title('Minimum Interpoint Distances (larger the better)')
    axs.set_xticks(list(i+1 for i in range(len(strategies))))
    axs.set_xticklabels(list(strategy.name for strategy in strategies))

    plt.show()

    assessmentic_data = [[t.measurement_set.measurements[measure.METRIC_WHOLE_MIN_PROJECTED_DISTANCE] for t in strategy.designs ] for strategy in strategies]
    fig, axs = plt.subplots(nrows=1, ncols=1, figsize=(9, 4))
    print(assessmentic_data)
    axs.violinplot(assessmentic_data,
                    showmeans=False,
                    showmedians=True)
    axs.set_title('Minimum Projected Distances (larger the better)')
    axs.set_xticks(list(i+1 for i in range(len(strategies))))
    axs.set_xticklabels(list(strategy.name for strategy in strategies))

    plt.show()

    assessmentic_data = [[t.measurement_set.measurements[measure.METRIC_AVG_MIN_PROJECTED_DISTANCE] for t in strategy.designs ] for strategy in strategies]
    fig, axs = plt.subplots(nrows=1, ncols=1, figsize=(9, 4))
    print(assessmentic_data)
    axs.violinplot(assessmentic_data,
                    showmeans=False,
                    showmedians=True)
    axs.set_title('Average Minimum Dimension Distances (larger the better)')
    axs.set_xticks(list(i+1 for i in range(len(strategies))))
    axs.set_xticklabels(list(strategy.name for strategy in strategies))

    plt.show()

In [8]:
def peek_into_results(strategies:List[DesignStrategyResults]):

    for strategy in strategies:
        print("\n\n-----------------------------------")
        print(f"Strategy: {strategy.name}")
        doe_tuple = strategy.designs[0]
        print("-Example Design Peak-------------------")
        print(doe_tuple.design.decoded_input_sets[0:5,:])
        print("-Measurements-------------------")
        print(doe_tuple.measurement_set.measurements)
        print("-Sub-Design Assessments----------------")
        print(doe_tuple.measurement_set.full_sub_design_measurements)
        

In [9]:
def plot_designs(strategies:List[DesignStrategyResults],design_index=0):
    
    for strategy in strategies:
        design = strategy.designs[design_index].design
        fig = plots.scatterplot_matrix(design.decoded_input_sets, list(design.input_set_map.keys()))
        fig.suptitle(strategy.name)

In [10]:
from raxpy.does.maxpro import optimize_design_with_sa

def add_maxpro_strategy_variations(stratgies:List[DesignStrategyResults]):
    s_len = len(stratgies)
    for i in range(s_len):
        base_strategy = stratgies[i]

        new_dsr = DesignStrategyResults(
            strategy=lambda x:x,
            name=f"{base_strategy.name}+MaxPro",
            designs=[]
        )
        for doe_tuple in base_strategy.designs:
            opt_design = optimize_design_with_sa(doe_tuple.design, encoding=EncodingEnum.NONE)
            measurement_set = measure.measure_with_all_metrics(opt_design, encoding=EncodingEnum.NONE)
            new_dsr.designs.append(DoeTuple(opt_design,measurement_set))
        stratgies.append(new_dsr)
        

# Assessment A: 3 Optional Floats

In [11]:
strategies = [
    DesignStrategyResults(lhs_doe.generate_design, "LHD-by-TreeTraversal",[]),
    DesignStrategyResults(lhs_doe.generate_seperate_designs_by_full_subspace_and_pool, "LHD-by-SubSpace-Projection",[]),
    DesignStrategyResults(random_doe.generate_design, "Random",[]),
    DesignStrategyResults(random_doe.generate_seperate_designs_by_full_subspace, "Random-by-SubSpace",[]),
    DesignStrategyResults(lhs_doe.generate_seperate_designs_by_full_subspace, "LHD-by-SubSpace",[]),
    DesignStrategyResults(lhs_doe.generate_design_with_projection, "LHD-Projection", []),
]

space = s.InputSpace(
    dimensions=[
        s.Float(id="x1", lb=0.0, ub=1.0, nullable=True),
        s.Float(id="x2", lb=0.0, ub=1.0, nullable=True),
        s.Float(id="x3", lb=0.0, ub=1.0, nullable=True),
    ]
)

target_sub_space_allocations = {
    tuple():0,
    ("x1",):1,
    ("x2",):1,
    ("x3",):1,
    ("x1","x2"):8,
    ("x1","x3"):8,
    ("x2","x3"):8,
    ("x1","x2","x3"):73,
}

small_target_sub_space_allocations = {
    tuple():0,
    ("x1",):1,
    ("x2",):1,
    ("x3",):1,
    ("x1","x2"):3,
    ("x1","x3"):3,
    ("x2","x3"):3,
    ("x1","x2","x3"):12,
}
c.assign_null_portions(s.create_level_iterable(space.children))

By default when creating dimensions, the target portion of values in a design to be null is unspecified. Creating a design without specifying these values, results in the whole design to have parameters. The following code assigns these portitions using a complexity analysis hueristic.

In [ ]:
plot_fullsubspace_target_portions(space,number_of_points=25)

In [ ]:
generate_designs(
    strategies, space, number_of_designs=4, number_of_points=25
)

In [ ]:
add_maxpro_strategy_variations(strategies)

In [ ]:
plot_designs(strategies,design_index =1)

In [ ]:
assessment_group = split_designs_by_subspace_allocations(strategies, space)
assessment_group.print_allocation_counts()
assessment_group.generate_allocation_point_differences(target_sub_space_allocations)

In [ ]:
# now lets reset and analyze without random
strategies = [
    DesignStrategyResults(lhs_doe.generate_design, "LHD-by-TreeTraversal",[]),
    # DesignStrategyResults(random_doe.generate_design, "Random",[]),
    DesignStrategyResults(lhs_doe.generate_seperate_designs_by_full_subspace_and_pool, "LHD-by-SubSpace-Projection",[]),
    DesignStrategyResults(random_doe.generate_seperate_designs_by_full_subspace, "Random-by-SubSpace",[]),
    DesignStrategyResults(lhs_doe.generate_seperate_designs_by_full_subspace, "LHD-by-SubSpace",[]),
    DesignStrategyResults(lhs_doe.generate_design_with_projection, "LHD-Projection", []),
]
generate_designs(
    strategies, space, number_of_designs=25,
    target_full_sub_space_portions=target_sub_space_allocations
)

In [ ]:
strategies[0].designs[0].design.input_sets

In [ ]:
strategies[0].designs[0].design.decoded_input_sets

In [ ]:
strategies[0].designs[0].measurement_set.full_sub_set_assessments

In [ ]:
strategies[0].designs[0].measurement_set.measurements

In [ ]:
f1 = plots.scatterplot_matrix(strategies[0].designs[0].design.decoded_input_sets, ["x1", "x2", "x3"])
f2 = plots.scatterplot_matrix(strategies[1].designs[0].design.decoded_input_sets, ["x1", "x2", "x3"])
f3 = plots.scatterplot_matrix(strategies[2].designs[0].design.decoded_input_sets, ["x1", "x2", "x3"])
f4 = plots.scatterplot_matrix(strategies[3].designs[0].design.decoded_input_sets, ["x1", "x2", "x3"])

In [ ]:
generate_basic_plots(strategies)

In [ ]:
measure.compute_whole_min_point_distance(strategies[1].designs[0].design,[],measure.EncodingEnum.NONE)

In [ ]:
# custom assessment plots
plot_sub_space_assessments(strategies, ("x1","x2","x3"), measure.METRIC_DISCREPANCY)
plot_sub_space_assessments(strategies, ("x1","x2"), measure.METRIC_DISCREPANCY)
plot_sub_space_assessments(strategies, ["x2","x3"], measure.METRIC_DISCREPANCY)
plot_sub_space_assessments(strategies, ["x1","x3"], measure.METRIC_DISCREPANCY)
plot_sub_space_assessments(strategies, ["x1"], measure.METRIC_DISCREPANCY)
plot_sub_space_assessments(strategies, ["x2"], measure.METRIC_DISCREPANCY)
plot_sub_space_assessments(strategies, ["x3"], measure.METRIC_DISCREPANCY)

# Assessment B: Basic Heirarchy

In [ ]:
strategies_bh = [
    DesignStrategyResults(lhs_doe.generate_design, "LHD-by-TreeTraversal",[]),
    DesignStrategyResults(lhs_doe.generate_seperate_designs_by_full_subspace_and_pool, "LHD-by-SubSpace-Projection",[]),
    #DesignStrategyResults(random_doe.generate_design, "Random",[]),
    
    DesignStrategyResults(random_doe.generate_seperate_designs_by_full_subspace, "Random-by-SubSpace",[]),
    DesignStrategyResults(lhs_doe.generate_seperate_designs_by_full_subspace, "LHD-by-SubSpace",[]),
    DesignStrategyResults(lhs_doe.generate_design_with_projection, "LHD-Projection", []),
]

space_bh = s.InputSpace(
    dimensions=[
        s.Float(id="x1", lb=0.0, ub=1.0, nullable=False),
        s.Float(id="x2", lb=0.0, ub=1.0, nullable=False),
        s.Float(id="x3", lb=0.0, ub=1.0, nullable=True, portion_null=(1/3)),
        s.Composite(id="x4", nullable=True, children=[
            s.Float(id="x4_1", lb=0.0, ub=1.0, nullable=False),
            s.Float(id="x4_2", lb=0.0, ub=1.0, nullable=True),
        ], portion_null=(1/3))
    ]
)

c.assign_null_portions(s.create_level_iterable(space_bh.children))
number_of_points = 30
plot_fullsubspace_target_portions(space_bh,number_of_points)

target_full_sub_space_portions_bh={
    ("x1","x2","x3"):17,
    ("x1","x2","x4","x4_1","x4_2"):14,
    ("x1","x2","x4","x4_1"):5,
    ("x1","x2","x3","x4","x4_1","x4_2"):45,
    ("x1","x2","x3","x4","x4_1"):14,
    ("x1","x2"):5
}
space_bh.dimensions[3]

In [ ]:
generate_designs(
    strategies_bh,
    space_bh,
    number_of_designs=20,
    number_of_points=number_of_points,
)

In [ ]:
add_maxpro_strategy_variations(strategies_bh)

In [ ]:
generate_basic_plots(strategies_bh)

# Assessment C: More complex hierarchy

In [ ]:

strategies_ch = [
    DesignStrategyResults(lhs_doe.generate_design, "LHD-by-TreeTraversal",[]),
    DesignStrategyResults(lhs_doe.generate_seperate_designs_by_full_subspace_and_pool, "LHD-by-SubSpace-Projection",[]),
    DesignStrategyResults(random_doe.generate_design, "Random",[]),
    DesignStrategyResults(lhs_doe.generate_seperate_designs_by_full_subspace, "LHD-by-SubSpace",[]),
    DesignStrategyResults(lhs_doe.generate_design_with_projection, "LHD-Projection", []),
]

space_ch = s.InputSpace(
    dimensions=[
        s.Float(id="x1", lb=0.0, ub=1.0),
        s.Float(
            id="x2",
            lb=0.0,
            ub=1.0,
            nullable=True,
            portion_null=1.0 / 10.0,
        ),
        s.Composite(
            id="x3",
            nullable=True,
            portion_null=1.0 / 7.0,
            children=[
                s.Float(
                    id="x4",
                    lb=0.0,
                    ub=1.0,
                    nullable=True,
                    portion_null=1.0 / 10.0,
                ),
                s.Float(
                    id="x5",
                    lb=0.0,
                    ub=1.0,
                    nullable=True,
                    portion_null=1.0 / 10.0,
                ),
            ],
        ),
        s.Variant(
            id="x6",
            nullable=True,
            portion_null=0.33,
            options=[
                s.Float(id="x7", lb=0.0, ub=1.0),
                s.Float(id="x8", lb=0.0, ub=1.0),
            ],
        ),
    ]
)

c.assign_null_portions(s.create_level_iterable(space_ch.children))
plot_fullsubspace_target_portions(space_ch)

In [ ]:
generate_designs(
    strategies_ch,
    space_ch,
    number_of_designs=20,
    number_of_points=number_of_points,
)

In [ ]:
generate_basic_plots(strategies_ch)

In [ ]:
plot_designs(strategies_ch,design_index =1)

In [ ]:
peek_into_results(strategies_ch)

In [ ]:
strategies_ch_2 = [
    DesignStrategyResults(lhs_doe.generate_design, "LHD-by-TreeTraversal",[]),
    DesignStrategyResults(lhs_doe.generate_seperate_designs_by_full_subspace_and_pool, "LHD-by-SubSpace-Projection",[]),
    DesignStrategyResults(random_doe.generate_design, "Random",[]),
    DesignStrategyResults(lhs_doe.generate_seperate_designs_by_full_subspace, "LHD-by-SubSpace",[]),
    DesignStrategyResults(lhs_doe.generate_design_with_projection, "LHD-Projection", []),
]
generate_designs(
    strategies_ch_2,
    space_ch,
    number_of_designs=20,
    number_of_points=30,
)

In [ ]:
peek_into_results(strategies_ch_2)

In [ ]:
generate_basic_plots(strategies_ch_2)

In [ ]:
plot_fullsubspace_target_portions(space_ch,number_of_points=30)

In [ ]:
plot_designs(strategies_ch_2,design_index =1)

In [ ]:
from scipy.stats.qmc import discrepancy

In [ ]:
discrepancy([[0.0],[0.5],[1.0]],method="WD")

In [ ]:
discrepancy([[0.0],[0.1],[1.0]],method="WD")

In [ ]:
discrepancy([[0.0],[0.001],[1.0]],method="WD")

In [ ]:
discrepancy([[0.0],[0.1],[0.05]],method="WD")

In [ ]:
discrepancy([[0.0],[0.1],[0.2],[0.3],[0.4],[0.5]],method="WD")

In [ ]:
discrepancy([[0.0],[0.19],[0.2],[0.3],[0.4],[0.5]],method="WD")

In [ ]:
discrepancy([[0.0],[0.19],[0.2],[0.21],[0.4],[0.5]],method="WD")

In [ ]:
discrepancy([[0.0],[0.19],[0.2],[0.21],[0.49],[0.5]],method="WD")

In [ ]:
discrepancy([[0.0],[0.01],[0.2],[0.21],[0.49],[0.5]],method="WD")

In [ ]:
help(discrepancy)

In [ ]:
discrepancy([[0.0],[0.19],[0.2],[0.21],[0.49],[0.5]], method="CD")

In [ ]:
discrepancy([[0.0],[0.1],[0.2],[0.3],[0.4],[0.5]], method="CD")

In [ ]:
1.0/ (4+1)